In [20]:
%matplotlib widget
from pathlib import Path

import numpy as np
from lotr import A_FISH, FIGURES_LOCATION, LotrExperiment, dataset_folders

from matplotlib import cm
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import mannwhitneyu
from tqdm import tqdm
import flammkuchen as fl
import scipy 
import statsmodels.api as sm

In [6]:
def monoExp(x, m, t, b):
    return m * np.exp(-t * x) + b

In [26]:
fig, ax = plt.subplots(7, 5, figsize=(8, 10), sharey=True, sharex=True)

i=0
for path in dataset_folders:
    r = i // 5
    c = np.mod(i, 5)
    
    exp = LotrExperiment(path)
    fs = exp.fs
    
    suit2p_data = fl.load(path / 'data_from_suite2p_unfiltered.h5')
    traces = fl.load(path / "filtered_traces.h5", "/detr")
    selected = fl.load(path / 'selected.h5')
    coords = suit2p_data['coords']
    
    n_lags = 1000
    num_traces = np.shape(traces)[1]
    ac_all = np.zeros((n_lags + 1, num_traces))
    all_tau = np.zeros(num_traces)
    ac_len = n_lags + 1
    xs = (1 / fs)*np.arange(ac_len)


    all_mag =  np.empty(num_traces)
    all_b =  np.empty(num_traces)

    p0 = (1, .1, 0) # start with values near those we expect
    for trace in range(num_traces):
        # Get autocoorrelation function 
        ac_all[:, trace] = sm.tsa.acf(traces[:,trace], nlags=n_lags)

        # Fit AC and get time constant 
        try:
            params, cv = scipy.optimize.curve_fit(monoExp, xs, ac_all[:, trace], p0)
            m, t, b = params    
            all_mag[i] = m
            all_b[i] = b
            tauSec = (1 / t) / fs
            all_tau[trace] = t
        except:
            all_tau[trace] = None

    ax[r, c].scatter(coords[:, 1], coords[:, 2], c=all_tau, cmap='hot', s=3, vmin=0, vmax=0.35)
    ax[r, c].axis('off')
    i += 1
    
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
fig, all_axs = plt.subplots(11,
    12,
    figsize=(9, 12),
    gridspec_kw=dict(
        top=0.88,
        right=1,
        bottom=0.01,
        hspace=0.3,
        wspace=0,
        width_ratios=[1, 1, 1, 0.3] * 3,
    ),
    sharey=True,
    sharex=True,
)
